# <center>Clase 5: POS Tagging</center>

#### ¿Qué es taguear? ¿Para qué nos sirve?

● Asignarle a cada palabra su clase gramatical (‘part of speech’).  
● Convertir un objeto sentence (lista de palabras) en una lista de tuplas
(word, tag).  
● Paso previo necesario antes casi cualquier otra tarea de más alto nivel:
lemmatizing, chunking, parsing, word sense disambiguation.  


Para esto vamos a utilizar un corpus, a fin de evaluar los taggers que vayamos desarrollando y, en algunos casos, también para entrenarlos.

En principio podemos decir que hay dos clases de corpus:
1. Corpus ‘crudos’: cualquier texto en formato digital, como los cuentos que usaron en clases anteriores.
2. Corpus anotados: contienen algún tipo de anotación lingüística (clase de palabra, sintaxis, entidades, valoración, etc.)

Para esta clase vamos a usar corpus con anotación de clase de palabra (part of speech, POS). En NLTK tenemos dos corpus con anotación de clase de palabra en español:

● CoNLL2002 (Conference on Computational Natural Language Learning)  
● CessEsp (corpus multilingüe de español y catalán)

En este encuentro vamos a trabajar con el CoNLL2002.


El formato del CoNLL2002 es bastante simple, cada archivo del corpus tiene una palabra por línea, junto con su POStag y su namedEntity tag. Las líneas vacías señalan límites entre oraciones, y las líneas que dicen '-DOCSTART-' marcan límites entre artículos (el corpus se compone de artículos periodísticos).  

Palabra POSTag namedEntityTag
    
Según SP O  
los DA O  
datos NC O  
difundidos AQ O  
hoy RG O  
por SP O  
Telefónica AQ B-ORG  
, Fc O  
la DA O  
empresa NC O  
ha VAI O  
impuesto VMP O  
en SP O  
Sao NC B-LOC  
Paulo VMI I-LOC  
una DI O  
marca NC O  
mundial AQ O  
en SP O  
la DA O  
expansión NC O  
de SP O  
redes NC O  
de SP O  
telefonía NC O  
fija AQ O  

#### ¿Cómo cargo un corpus en NLTK?

In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import cess_esp
from nltk.corpus import conll2002

In [2]:
print(conll2002.readme())

These files contain the train and test data for for the three parts of 
the CoNLL-2002 shared task:

   esp.testa: Spanish test data for the development stage
   esp.testb: Spanish test data
   esp.train: Spanish train data
   ned.testa: Dutch test data for the development stage
   ned.testb: Dutch test data
   ned.train: Dutch train data

All data files contain a single word per line with it associated 
named entity tag in the IOB2 format (Tjong Kim Sang and Veenstra,
EACL 1999). Sentence breaks are encoded by empty lines. Additionally
the Dutch data contains non-checked part-of-speech tags generated
by the MBT tagger (Daelemans et.al., WVLC 1996). In the Dutch data
article boundaries have been marked by a special tag (-DOCSTART-).

Associated url: http://lcg-www.uia.ac.be/conll2002/ner/


NOTES

* Files in these directories may only be used for research
  applications in the context of the CoNLL-2002 shared task.
  No permission is given for usage other applications especially
  not 

In [3]:
corpus_words = conll2002.words('esp.train')
corpus_tagged_words = conll2002.tagged_words('esp.train')
corpus_sents = conll2002.sents('esp.train')

Cuando cargamos un corpus utilizando nltk.corpus() disponemos de ciertos métodos propios de estos objetos:
   
● corpus.fileids()<br> 
● corpus.raw()  <br>
● corpus.words()  <br>
● corpus.sents()  <br>
● corpus.tagged_sents()<br> 


In [4]:
print(type(corpus_words))
print(corpus_words[:10])
print(corpus_sents[5])

<class 'nltk.collections.LazyConcatenation'>
[u'Melbourne', u'(', u'Australia', u')', u',', u'25', u'may', u'(', u'EFE', u')']
[u'Por', u'su', u'parte', u',', u'el', u'Abogado', u'General', u'de', u'Victoria', u',', u'Rob', u'Hulls', u',', u'indic\xf3', u'que', u'no', u'hay', u'nadie', u'que', u'controle', u'que', u'las', u'informaciones', u'contenidas', u'en', u'CrimeNet', u'son', u'veraces', u'.']


#### Vamos a inspeccionar un poco el corpus

In [5]:
def corpus_info():
    print('Palabras: ' + str(len(corpus_words)))
    print('Oraciones: ' + str(len(corpus_sents)))
    print('Lexico: ' + str(len(set(corpus_words))))

corpus_info()

def tagset_info():
    tags = [str(tag) for (word, tag) in corpus_tagged_words]
    tagset = set(tags)
    print("Tagset", tagset)
    print("Cantidad de elementos en el tagset", len(tagset))

tagset_info()

Palabras: 264715
Oraciones: 8323
Lexico: 26099
('Tagset', set(['DN', 'Fpa', 'VSP', 'Fit', 'DI', 'DD', 'DA', 'Fpt', 'Fia', 'DT', 'DP', 'Fp', 'PP', 'Fs', 'Ft', 'PT', 'Fx', 'Fz', 'NC', 'Fat', 'I', 'Fc', 'Fd', 'Fe', 'PD', 'Fg', 'NP', 'RN', 'PI', 'PN', 'Y', 'PR', 'P0', 'VAI', 'VAN', 'VAM', 'CC', 'AO', 'AQ', 'RG', 'VSS', 'PX', 'CS', 'VAS', 'VAP', 'Z', 'Fh', 'VMN', 'Faa', 'VMM', 'VMI', 'VMG', 'SP', 'VSM', 'VSN', 'VSG', 'VSI', 'VMS', 'VMP']))
('Cantidad de elementos en el tagset', 59)


#### Frecuencia y frecuencia acumulada de las etiquetas

![alt text](tagset_dist.jpg "Frecuencia acumulada de tags")

![alt text](tagsettable.jpg "Frecuencia acumulada de tags")


# Taggers: modelos entrenables y no entrenables  

A partir de acá, vamos a empezar a introducir, modelar y evaluar una serie de taggers. <br>

Con 'taggers' nos referimos a algoritmos que toman como input un objeto sentence (una lista de palabras) y devuelven una lista de tuplas (palabra, tag). <br>

Cualquiera de estos taggers no toma como input un texto 'crudo' (por ejemplo una string, o un archivo en formato .txt) sino un texto ya tokenizado.

        -> Proceso: transformar una lista de palabras a una lista de tuplas (word, tag)  
        -> Preproceso: tokenizar la oración 
        
El módulo tokenize de nltk nos permite usar una función 'word_tokenize', que es un tokenizador genérico para cualquier idioma.

In [ ]:
from nltk.tokenize import word_tokenize

sentence = u'Arriba las manos! Llegó el estado.'
dummy_tokenized = sentence.split(' ')
print(dummy_tokenized)
tokenized = word_tokenize(sentence)
print(tokenized)

Al igual que los corpus, todos los taggers generados en NLTK tienen algunos métodos en común:   
        ● tag()  
        ● tag_sents()  
        ● untag()  
        ● evaluate() 


Para trabajar con taggers entrenables es necesario dividir nuestros datos (en nuestro caso, el corpus con oraciones postaggeadas) en dos subconjuntos, uno de entrenamiento y otro de testeo. Lo que vamos hacer cuando trabajemos con taggers entrenables es generar un modelo a partir del conjunto de entrenamiento y evaluarlo posteriormente a partir del conjunto de test.

![alt text](traintest.png "Train and test sets")

In [6]:
train_tagged_sents = conll2002.tagged_sents('esp.train')
test_tagged_sents = conll2002.tagged_sents('esp.testa')

print(train_tagged_sents[5])

[(u'Por', u'SP'), (u'su', u'DP'), (u'parte', u'NC'), (u',', u'Fc'), (u'el', u'DA'), (u'Abogado', u'NC'), (u'General', u'AQ'), (u'de', u'SP'), (u'Victoria', u'NC'), (u',', u'Fc'), (u'Rob', u'NC'), (u'Hulls', u'AQ'), (u',', u'Fc'), (u'indic\xf3', u'VMI'), (u'que', u'CS'), (u'no', u'RN'), (u'hay', u'VAI'), (u'nadie', u'PI'), (u'que', u'PR'), (u'controle', u'VMS'), (u'que', u'CS'), (u'las', u'DA'), (u'informaciones', u'NC'), (u'contenidas', u'AQ'), (u'en', u'SP'), (u'CrimeNet', u'NC'), (u'son', u'VSI'), (u'veraces', u'AQ'), (u'.', u'Fp')]


¿Qué sucedería si emplearamos los mismos datos para entrenar y para evaluar el modelo?

## 1. DEFAULT TAGGER

In [7]:
from nltk.tag import DefaultTagger

In [8]:
# Definimos cuál es el tag mas probable
tags = [tag for (word, tag) in corpus_tagged_words]
default_tag = nltk.FreqDist(tags).max()

# Asignamos ese tag a cualquier token
default_tagger = DefaultTagger(default_tag)

print(default_tag)
print("Test", default_tagger.tag(['Hola', 'Mundo']))
print("Default", default_tagger.evaluate(test_tagged_sents))

NC
('Test', [('Hola', u'NC'), ('Mundo', u'NC')])
('Default', 0.24178523515295808)


Aunque parezca trivial e innecesario, lo que acabamos de hacer nos va a ser útil en dos sentidos:  

● Establecer un baseline a partir del cual juzgar la performance del resto de los taggers  
● Como vamos a ver más adelante, si otro tagger no puede asignarle ningún tag a cierto token, es posible utilizar este default_tagger para que asigne el tag más probable del corpus.

# 2. REGEX TAGGER

La clase RegexpTagger() toma como argumento una lista de tuplas, en la que el primer elemento de cada
tupla es una expresión regular y el segundo elemento es el tag que se le va a asignar a las palabras que matchéen con ese pattern.  


In [10]:
from nltk.tag import RegexpTagger

patterns = [
 (r'.*ing$', 'VBG'),                # gerunds
 (r'.*ed$', 'VBD'),                 # simple past
 (r'.*es$', 'VBZ'),                 # 3rd singular present
 (r'.*ould$', 'MD'),                # modals
 (r'.*\'s$', 'NN$'),                # possessive nouns
 (r'.*s$', 'NNS'),                  # plural nouns
 (r'^-?[0-9]+(.[0-9]+)?$', 'CD'),   # cardinal numbers
 (r'.*', 'NC')                      # nouns (default)
 ]

'''
patterns = [
 (ur'.*ción$', 'NC')
]
'''

regexp_tagger = nltk.RegexpTagger(patterns)

In [11]:
print(regexp_tagger.tag(corpus_sents[3]))
print("Regex tagger", regexp_tagger.evaluate(test_tagged_sents))

[(u'La', 'NC'), (u'petici\xf3n', 'NC'), (u'del', 'NC'), (u'Abogado', 'NC'), (u'General', 'NC'), (u'tiene', 'NC'), (u'lugar', 'NC'), (u'despu\xe9s', 'NNS'), (u'de', 'NC'), (u'que', 'NC'), (u'un', 'NC'), (u'juez', 'NC'), (u'del', 'NC'), (u'Tribunal', 'NC'), (u'Supremo', 'NC'), (u'del', 'NC'), (u'estado', 'NC'), (u'de', 'NC'), (u'Victoria', 'NC'), (u'(', 'NC'), (u'Australia', 'NC'), (u')', 'NC'), (u'se', 'NC'), (u'viera', 'NC'), (u'forzado', 'NC'), (u'a', 'NC'), (u'disolver', 'NC'), (u'un', 'NC'), (u'jurado', 'NC'), (u'popular', 'NC'), (u'y', 'NC'), (u'suspender', 'NC'), (u'el', 'NC'), (u'proceso', 'NC'), (u'ante', 'NC'), (u'el', 'NC'), (u'argumento', 'NC'), (u'de', 'NC'), (u'la', 'NC'), (u'defensa', 'NC'), (u'de', 'NC'), (u'que', 'NC'), (u'las', 'NNS'), (u'personas', 'NNS'), (u'que', 'NC'), (u'lo', 'NC'), (u'compon\xedan', 'NC'), (u'pod\xedan', 'NC'), (u'haber', 'NC'), (u'obtenido', 'NC'), (u'informaci\xf3n', 'NC'), (u'sobre', 'NC'), (u'el', 'NC'), (u'acusado', 'NC'), (u'a', 'NC'), (u'tr

# 3. AFFIX TAGGER

El Affix tagger va a funcionar de manera similar al regexpTagger, solo que en vez de definir nosotros un conjunto de patterns, el tagger los va a aprender automáticamente basándose en afijos de una medida fija (uno, dos, tres, n caracteres), tanto al comienzo como al final de la palabra.  


In [12]:
from nltk.tag import AffixTagger

tagger = AffixTagger(train_tagged_sents)
print("Affix tagger", tagger.evaluate(test_tagged_sents))


('Affix tagger', 0.28762541806020064)


Parameters:  
● affix_length – The length of the affixes that should be considered during training and tagging. Use negative numbers
for suffixes. (default = -3)  
● min_stem_length – Any words whose length is less than min_stem_length+abs(affix_length) will be assigned a tag of
None by this tagger. (default = 2)  


In [13]:
suffix_tagger = AffixTagger(train_tagged_sents, affix_length=-2)
print("Affix tagger II", suffix_tagger.evaluate(test_tagged_sents))

('Affix tagger II', 0.2955425807304953)


# 4. LOOKUP TAGGER

In [14]:
fd = nltk.FreqDist(corpus_words)
cfd = nltk.ConditionalFreqDist(corpus_tagged_words)
most_freq_words = [word for word, tag in fd.most_common(500)]
print(most_freq_words[:10])

[u'de', u',', u'la', u'que', u'.', u'el', u'en', u'"', u'y', u'a']


Almacena el tag más usual para las n palabras más usuales.


In [15]:
likely_tags = dict((word, cfd[word].max()) for word in most_freq_words)
lookup_tagger = nltk.UnigramTagger(model=likely_tags)
print("Look up", lookup_tagger.evaluate(test_tagged_sents))

('Look up', 0.6125692043157039)


# 5. BACKOFF

In [16]:
backoff_tagger = nltk.UnigramTagger(model=likely_tags, backoff=regexp_tagger)

#sent = corpus_sents[3]
#sent = nltk.tokenize.word_tokenize(u'Nunca seré policía')
sent = ['Nunca', 'seré', 'policía']
print(backoff_tagger.tag(sent))
print("Look up", backoff_tagger.evaluate(test_tagged_sents))

[('Nunca', 'NC'), ('ser\xc3\xa9', 'NC'), ('polic\xc3\xada', 'NC')]
('Look up', 0.7401507851028853)


# Evaluación de los modelos

### ¿Qué palabras tagueamos mal?

In [17]:
def bad_tagged_words(tagger):
    bad_tagged = []
    test_tags = [(word, tag) for sent in corpus_sents for (word, tag) in tagger.tag(sent)]
    gold_tags = [(word, tag) for sent in train_tagged_sents for (word, tag) in sent]
    for i, item in enumerate(test_tags):
        if item!=gold_tags[i]:
            # palabra, predicted tag, expected tag
            bad_tagged.append((item[0], item[1], gold_tags[i][1]))
    return bad_tagged

bad = bad_tagged_words(backoff_tagger)
print(bad[:10])

[(u'Melbourne', 'NC', u'NP'), (u'Australia', 'NC', u'NP'), (u'General', 'NC', u'AQ'), (u'Daryl', 'NC', u'VMI'), (u'Williams', 'NNS', u'NC'), (u'subray\xf3', 'NC', u'VMI'), (u'tomar', 'NC', u'VMN'), (u'proteger', 'NC', u'VMN'), (u'judicial', 'NC', u'AQ'), (u'australiano', 'NC', u'AQ')]


### ¿Qué palabras tagueamos mal con más frecuencia?

In [18]:
from collections import Counter
bad_tagged_words = [i[0] for i in bad]
counter = Counter(bad_tagged_words)
sorted_counter = sorted(counter.items(), key=lambda kv: kv[1], reverse=True)
print(sorted_counter[:20])

[(u'que', 2910), (u'lo', 141), (u'Jos\xe9', 85), (u'Madrid', 70), (u'otros', 51), (u'pasado', 49), (u'tarde', 46), (u'Carlos', 45), (u'todo', 45), (u'/', 43), (u'autoridades', 43), (u'Central', 43), (u'18', 43), (u'algunas', 43), (u'propio', 43), (u'LA', 42), (u'todav\xeda', 42), (u'cuya', 42), (u'responsables', 42), (u'hecho', 42)]


### Matriz de confusión

In [ ]:
#import seaborn as sn
#import pandas as pd
#import matplotlib.pyplot as plt
#import sklearn as skl

selectedTags = [i for i,j in nltk.FreqDist(tags).most_common(15)]
def confusion_matrix(tagger):
    test_tags = [tag for sent in corpus_sents for (word, tag) in tagger.tag(sent)]
    gold_tags = [tag for (word, tag) in corpus_tagged_words]
    labels = set(gold_tags)
    cm = skl.metrics.confusion_matrix(gold_tags, test_tags, 
                                     labels=selectedTags)
    cm = cm / cm.astype(np.float).sum(axis=1)
    return cm

#cm = confusion_matrix(backoff_tagger)
#df_cm = pd.DataFrame(cm, index = selectedTags, columns = selectedTags)
#plt.figure(figsize = (7,7))
#sn.heatmap(df_cm, annot=False, cmap=plt.cm.Blues)
#plt.show()

![alt text](heatmap.png)

¿Cuál es el límite teórico de un look-up tagger, considerando un corpus que contenga todas las posibles ocurrencias de un
lenguaje?  

a. En el juego de la vida vos podés triunfar.  
b. Juego al futbol todos los martes.  
c. Como agua para chocolate.  
d. Me como una pizza entera.  

In [19]:
from nltk.text import Text
my_text = Text(corpus_words)
concordance = my_text.concordance('juego')
print(concordance)

Displaying 25 of 25 matches:
n unánimes los juicios favorables al juego desarrollado por el equipo ganador ,
 fue " soberbio " por la calidad del juego y un estadio divido entre las dos af
l que dicen los diarios no mostró se juego brillante que hizo contra el Barcelo
dondo y McManaman . Tras destacar el juego de Raul , " La Gazzetta dello Sport 
te torneo para pulir sus esquemas de juego y analizar la situación de sus jugad
 , Azócar pegó un cuadrangular en el juego que perdió Oaxaca por 14-9 ante Pueb
identa del comité organizador de los juego olímpicos de Atenas 2004 , Miki Tzav
ión en el lenguaje , pero también un juego divertido " y para la quebequesa Dor
ka " es sencilla " : " jugar nuestro juego e intentar ganar y si yo marco o no 
midos y no han podido desarrollar el juego que llevan dentro . Además , quiero 
 va a dejar la piel en el terreno de juego defendiendo los colores . El que no 
recalcó la importancia del trofeo en juego . " Ganar la Copa no es una disculpa
a a nuestra

# Performance del look-up tagger con diferentes tamaños de diccionario

In [20]:
def performance(cfd, wordlist):
    lt = dict((word, cfd[word].max()) for word in wordlist)
    baseline_tagger = nltk.UnigramTagger(model=lt, backoff=nltk.DefaultTagger('NN'))
    return backoff_tagger.evaluate(train_tagged_sents)

def display():
    import pylab
    #Distribución de frecuencias de las palabras
    freq_dist = nltk.FreqDist(corpus_words)
    #Muestra la distribución de tags para cada palabra
    cfd = nltk.ConditionalFreqDist(corpus_tagged_words)
    sizes = 2 ** pylab.arange(16)
    perfs = [performance(cfd, [x[0] for x in freq_dist.most_common(size)]) for size in sizes]
    pylab.plot(sizes, perfs, '-bo')
    pylab.title('Performance del Look-up tagger')
    pylab.xlabel('Size')
    pylab.ylabel('Performance')
    pylab.show()

#display()

![alt text](lookup.jpg)

# N-gramas

Un n-grama es un subsecuencia de n items. En ese sentido, la clase de NLTK BigramTagger va a mirar dos items (la palabra que queremos taguear y la anterior), mientras que la clase TrigrammTagger va a decidir qué tag asignar teniendo en cuenta tres items. Estos dos taggers van a ser buenos lidiando con una de las características que tienen los lenguajes naturales, el hecho de que el postag de una palabra depende del contexto en que esa palabra se encuentre.

Muchas palabras tienen diferentes postags de acuerdo a cómo sean usadas, como en el ejemplo que vimos más arriba de 'juego' que puede ser utilizada tanto como nombre común (su acepción más usual en nuestro corpus) y como verbo presente en primera persona del singular.

La idea detrás de los taggers de n-gramas es que mirando las palabras previas a la palabra target podemos inferir con mayor precisión cuál es el tag adecuado que debemos asignar.

# 6. UNIGRAM TAGGER

In [21]:
from nltk.tag import UnigramTagger

unitagger = UnigramTagger(train_tagged_sents)
print(unitagger.tag(word_tokenize(u'El perro me mordió fiero')))
print("Unigram", unitagger.evaluate(test_tagged_sents))

# WITH BACKOFF
unitagger = UnigramTagger(train_tagged_sents, backoff=default_tagger)
print(unitagger.tag(word_tokenize(u'El perro me mordió fiero')))
print("Unigram con backoff", unitagger.evaluate(test_tagged_sents))

[(u'El', u'DA'), (u'perro', None), (u'me', u'PP'), (u'mordi\xf3', None), (u'fiero', None)]
('Unigram', 0.8720971978156945)
[(u'El', u'DA'), (u'perro', u'NC'), (u'me', u'PP'), (u'mordi\xf3', u'NC'), (u'fiero', u'NC')]
('Unigram con backoff', 0.9030667195737203)


# 7. BIGRAM & TRIGRAM TAGGERS

In [22]:
from nltk.tag import BigramTagger, TrigramTagger

bitagger = BigramTagger(train_tagged_sents)
print("Bigram", bitagger.evaluate(test_tagged_sents))

tritagger = TrigramTagger(train_tagged_sents)
print("Trigram", tritagger.evaluate(test_tagged_sents))

('Bigram', 0.2264232942199044)
('Trigram', 0.16926478090811178)


Sparse data problem  
● ¿Por qué baja la accuracy?  
● ¿Qué pasa cuando no puede taguear una palabra nueva?  


In [23]:
sentence = nltk.tokenize.word_tokenize(u'Fue a la guerra')
sentence2 = nltk.tokenize.word_tokenize(u'Superman fue a la guerra')

print(bitagger.tag(sentence))
print(bitagger.tag(sentence2))

[(u'Fue', u'VSI'), (u'a', u'SP'), (u'la', u'DA'), (u'guerra', u'NC')]
[(u'Superman', None), (u'fue', None), (u'a', None), (u'la', None), (u'guerra', None)]


# MORE BACKOFF

In [24]:
def more_backoff(train_tagged_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_tagged_sents, backoff=backoff)
        return backoff

backoff_tagger = more_backoff(train_tagged_sents, [UnigramTagger, BigramTagger, TrigramTagger], backoff=default_tagger)
print("More backoff", backoff_tagger.evaluate(test_tagged_sents))

('More backoff', 0.9030667195737203)


# Todos somos héroes anónimos

In [25]:
def anonimize_NP(corpus):
    ANONYMOUS = "anonymous"
    new = []
    for s in corpus:
        for i, (w, tag) in enumerate(s):
            if tag == u"NP":  # NP = proper noun in Parole tagset.
                s[i] = (ANONYMOUS, u"NP")
        new.append(s)
    return new

anon_train_tagged_sents = anonimize_NP(train_tagged_sents)
anon_test_tagged_sents = anonimize_NP(test_tagged_sents)

anon_tagger = more_backoff(anon_train_tagged_sents, [UnigramTagger, BigramTagger, TrigramTagger], backoff=default_tagger)
print("Anon tagger", anon_tagger.evaluate(anon_test_tagged_sents))

('Anon tagger', 0.9097367874081213)


# Escritura automática a partir de bigramas

In [ ]:
from nltk.corpus import PlaintextCorpusReader
from random import choice
import os


def max_model(cfdist, word, num=15):
    autotext = ""
    for i in range(num):
        autotext += word + " "
        word = cfdist[word].max()
    return autotext


def choice_model(cfdist, word, num=15):
    autotext = ""
    for i in range(num):
        autotext += word + " "
        wordlist = list(cfdist[word])
        #print(wordlist)
        word = choice(wordlist)
    return autotext


def poem(text, cfd, model, lines=10, num=10):
    poem = ''
    for i in range(lines):
        word = choice(text)
        autotext = model(cfd, word, num)
        poem += autotext + '\n'
    return poem

corpus = PlaintextCorpusReader(os.getcwd(), "[a-zA-Z0-9]*.txt")
text = corpus.words('martinfierro.txt')
bigrams = nltk.bigrams(text)
cfd = nltk.ConditionalFreqDist(bigrams)
autotext1 = max_model(cfd, 'Cuando')
autotext2 = choice_model(cfd,"Cuando")

print(autotext1)
print('\n')
print(poem(text, cfd, max_model))
print('\n')
print(poem(text, cfd, choice_model))


# Pickles

<img src="rick.png" alt="Drawing" style="width: 200px;"/>

Pickles es un módulo de python que nos permite serializar objetos, es decir, guardar en un archivo cualquier objeto que hayamos instanciado en python.

Los taggers que vienen a continuación requieren un entrenamiento, por lo que vamos a tardar bastante tiempo en generar el tagger. Por eso, una vez entrenados resulta conveniente almacenarlos en un archivo usando pickle, para poder reutilizarlos posteriormente sin necesidad de re-entrenarlos.

Tiene dos métodos básicos: dump() para guardar un objeto en un archivo y load() para cargarlo.

In [27]:
import pickle

dummyVariable = "Nunca seré policía, de provincia ni de capital"
# Serializo el objeto dummyVariable
pickle.dump(dummyVariable, open("fuck.p", "wb"))

# Lo cargo en una nueva variable
loadVariable = pickle.load(open("fuck.p", "rb"))
print(loadVariable)

Nunca seré policía, de provincia ni de capital


# 8. BRILL TAGGER

El Brill tagger es un método inductivo para taggear. Fue desarrollado en 1993 por Eric Brill en su tesis de doctorado (https://dl.acm.org/citation.cfm?doid=974499.974526). <br>

Puede ser descripto como un tagger transformacional basado en minimizar la cantidad de errores. Vendría a ser una forma de aprendizaje supervisado que intenta minimizar el error a partir de un proceso transformacional, en el sentido de que cada tag es asignado a cada palabra y luego es revisado (y potencialmente cambiado) a partir de un set de reglas predefinidas que fueron inferidas del corpus. <br>

Aplicando iterativamente estas reglas, modificando los tags incorrectos, logra aumentar la precisión de cualquiera de los tags que vimos anteriormente. Las reglas que genera automáticamente permiten inferir información valiosa, como son las reglas morfosintácticas de combinación de las palabras, que luego son utilizadas en el proceso de tagueo.

Es decir que, una vez que a cada palabra se le ha asignado un tag provisional, una serie de reglas contextuales son aplicadas iterativamente, con el objetivo de "corregir" tags erroneos, a partir del exámen de pequeñas cantidades de contexto.

tag1 → tag2 IF Condition
1. Utiliza otro tagger para asignarle tags provisorios a cada palabra.  
2. Aplica iterativamente reglas basadas en el contexto.  
3. A cada regla se le asigna un puntaje (errores que corrige - errores que produce).  
4. Se queda con las reglas que maximizan la performance del tagger.
5. Las aplica a todo el análisis.  

Utiliza dos argumentos:  
● Tagger  
● Lista de templates



In [28]:
from nltk.tag import brill, brill_trainer

def train_brill_tagger(initial_tagger, train_sents, **kwargs):
    templates = [
        brill.Template(brill.Pos([-1])),
        brill.Template(brill.Pos([1])),
        brill.Template(brill.Pos([-2])),
        brill.Template(brill.Pos([2])),
        brill.Template(brill.Pos([-2, -1])),
        brill.Template(brill.Pos([1, 2])),
        brill.Template(brill.Pos([-3, -2, -1])),
        brill.Template(brill.Pos([1, 2, 3])),
        brill.Template(brill.Pos([-1]), brill.Pos([1])),
        brill.Template(brill.Word([-1])),
        brill.Template(brill.Word([1])),
        brill.Template(brill.Word([-2])),
        brill.Template(brill.Word([2])),
        brill.Template(brill.Word([-2, -1])),
        brill.Template(brill.Word([1, 2])),
        brill.Template(brill.Word([-3, -2, -1])),
        brill.Template(brill.Word([1, 2, 3])),
        brill.Template(brill.Word([-1]), brill.Word([1])),
    ]
    trainer = brill_trainer.BrillTaggerTrainer(initial_tagger, templates, deterministic=True, trace=True)

    return trainer.train(train_sents, max_rules=20, **kwargs)

initial_tagger = more_backoff(train_tagged_sents, [UnigramTagger, BigramTagger, TrigramTagger], backoff=default_tagger)
#BrillTagger = train_brill_tagger(initial_tagger, train_tagged_sents)
#pickle.dump(BrillTagger, open("BrillTagger.p", "wb"))
BrillTagger = pickle.load(open("BrillTagger.p", "rb"))

print("\n\n")
for rule in BrillTagger.rules(): print(str(rule.format('verbose')).encode('utf-8'))
print("\n\n")
print(BrillTagger.train_stats())
print("\n\n")
print("Brill tagger", BrillTagger.evaluate(test_tagged_sents))




PR -> CS if the Pos of words i-2...i-1 is "VMI"
PR -> CS if the Pos of the following word is "SP"
PR -> CS if the Pos of the following word is "Fe"
AQ -> VMP if the Pos of the preceding word is "VAI"
DA -> PP if the Pos of the following word is "VMI"
PR -> CS if the Pos of the preceding word is "SP", and the Pos of the following word is "DA"
DI -> PI if the Pos of the following word is "SP"
PR -> CS if the Pos of the preceding word is "CS"
PR -> CS if the Pos of the preceding word is "RG"
NP -> NC if the Pos of words i-2...i-1 is "SP"
AQ -> NC if the Pos of the preceding word is "SP", and the Pos of the following word is "AQ"
DN -> PN if the Pos of the following word is "SP"
PR -> CS if the Pos of the following word is "VMN"
PR -> CS if the Pos of the preceding word is "VMN"
DI -> PI if the Pos of the following word is "Fc"
AQ -> NC if the Pos of the preceding word is "Fc", and the Pos of the following word is "AQ"
AQ -> NC if the Pos of the preceding word is "DA", and the Pos of th

# Taggers basados en clasificadores automáticos

El postagging puede ser entendido como un problema de clasificación, en el que debemos clasificar ciertos inputs (tokens) entre una lista de categorías posibles (tags). Para automatizar esta clasificación podemos valernos de modelos estadísticos. Estos van a predecir las probabilidades de cada tag para cada uno de los inputs a partir de un conjunto de variables independientes que nosotros definamos.

2 componentes:  
● Feature extractor  
● Machine learning algorithm  

Clasificación supervisada:  
● Asigna una etiqueta para determinado input  
● Basándose en un corpora de entrenamiento que contiene las etiquetas correctas. 

![alt text](1.jpg)


In [ ]:
# Este es el source code de NLTK, vamos a intentar leer la función de feature_detector que utiliza:

from nltk.tag.sequential import ClassifierBasedTagger

class ClassifierBasedPOSTagger(ClassifierBasedTagger):
    """
    A classifier based part of speech tagger.
    """

    def feature_detector(self, tokens, index, history):
        word = tokens[index]
        if index == 0:
            prevword = prevprevword = None
            prevtag = prevprevtag = None
        elif index == 1:
            prevword = tokens[index-1].lower()
            prevprevword = None
            prevtag = history[index-1]
            prevprevtag = None
        else:
            prevword = tokens[index-1].lower()
            prevprevword = tokens[index-2].lower()
            prevtag = history[index-1]
            prevprevtag = history[index-2]

        if re.match('[0-9]+(\.[0-9]*)?|[0-9]*\.[0-9]+$', word):
            shape = 'number'
        elif re.match('\W+$', word):
            shape = 'punct'
        elif re.match('[A-Z][a-z]+$', word):
            shape = 'upcase'
        elif re.match('[a-z]+$', word):
            shape = 'downcase'
        elif re.match('\w+$', word):
            shape = 'mixedcase'
        else:
            shape = 'other'

        features = {
            'prevtag': prevtag,
            'prevprevtag': prevprevtag,
            'word': word,
            'word.lower': word.lower(),
            'suffix3': word.lower()[-3:],
            'suffix2': word.lower()[-2:],
            'suffix1': word.lower()[-1:],
            'prevprevword': prevprevword,
            'prevword': prevword,
            'prevtag+word': '%s+%s' % (prevtag, word.lower()),
            'prevprevtag+word': '%s+%s' % (prevprevtag, word.lower()),
            'prevword+word': '%s+%s' % (prevword, word.lower()),
            'shape': shape,
            }
        return features

# 9. Naive Bayes Classiffier

Este clasificador 'ingenuo' se basa en el teorema de Bayes:

![alt text](bayes.jpeg)

1. Calcula la probabilidad previa de cada tag, chequeando la etiqueta de cada uno en el corpus de entrenamiento.  
2. Cada rasgo contribuye “votando en contra” de aquellas etiquetas que no co-ocurren con él.  

<center>P(features, label) = P(label) × P(features|label)</center>

![alt text](5.jpg)


In [29]:
from nltk.tag.sequential import ClassifierBasedPOSTagger

#NaiveBayesTagger = ClassifierBasedPOSTagger(train=train_tagged_sents)
#pickle.dump(NaiveBayesTagger, open("NaiveBayesTagger.p", "wb"))
NaiveBayesTagger = pickle.load(open("NaiveBayesTagger.p", "rb"))
print("Bayes Classifier", NaiveBayesTagger.evaluate(test_tagged_sents))

('Bayes Classifier', 0.9241161687735011)


## Conclusión:  

●Los taggers basados en clasificadores automáticos son los que tienen mejor performance, pero también son los más lentos.  
●Si necesitamos un modulo rápido, lo mejor es usar un Bril Tagger precedido por una cadena de taggers basados en n-gramas.  

--> Brill tagger es un modelo más robusto y económico.


El desarrollo de taggers tuvo un rol central en el crecimiento de los enfoques estadísticos en el procesamiento del lenguaje natural.  
En los comienzos de la década de los 90' la performance de los taggers estadísticos brindó una prueba certera de que se podía resolver al menos una pequeña porción del entendimiento linguistico sin necesidad de acceder a un conocimiento más profundo de la estructura gramatical como es la sintaxis.


# Lematización

Lematizar es asignarle a cada palabra su forma de diccionario. Al igual que sucede con los algoritmos de stemming, es un proceso sumamente util para reducir la dimensionalidad de los datos y así mejorar tanto la eficiencia y como la eficacia de cualquier algoritmo de más alto nivel (un clasificador, por ejemplo).

Una vez que tenemos la clase de palabra de cada forma léxica, si contamos con un diccionario de formas y lemmas posibles (como el que nos provee freeling ;) ) lematizar se convierte en algo bastante trivial.

En el archivo dicc.src tenemos el diccionario de Freeling. Está organizado de esta manera:

promedio promediar VMIP1S0 promedio NCMS000 <br>
que que CS que PR0CN000 <br>
no no NCMS000 no RN <br>
noble noble AQ0CS0 noble NCCS000 <br>

El primer ítem es una forma léxica. A partir de ahí puede haber cualquier cantidad de pares lemma/tag, tantos como posibles tags tenga la palabra.

[Acá](/edit/lematizador.py) está el código de un lematizador bastante trivial que armé. Lo que hace es taguear el texto que querramos lematizar y luego buscar en el diccionario de freeling a qué lemma de esa forma léxica le corresponde ese tag. Para que la búsqueda sea más rápida, convertimos el archivo de texto en un diccionario de python y lo guardé en el archivo freeling_dicc.p

In [30]:
from lematizador import Lemmatizer

myLemmatizer = Lemmatizer(NaiveBayesTagger, debug=True)
myLemmatizer.build_dicc()

In [31]:
sample = u"Nunca te creí. Las calles de Buenos Aires tienen esas casas azules."
myLemmatizer.process(sample)

nunca
Pares de lemma/tag posibles:
[(u'nunca', u'RG')]
Tag elegido:
RG


te
Pares de lemma/tag posibles:
[(u'te', u'P02CS000'), (u'te', u'PP2CS000')]
Tag elegido:
PP2CS000


las
Pares de lemma/tag posibles:
[(u'el', u'DA0FP0'), (u'la', u'NCMP000'), (u'lo', u'PP3FPA00')]
Tag elegido:
DA0FP0


calles
Pares de lemma/tag posibles:
[(u'callar', u'VMSP2S0'), (u'calle', u'NCFP000')]
Tag elegido:
NCFP000


de
Pares de lemma/tag posibles:
[(u'de', u'NCFS000'), (u'de', u'SPS00')]
Tag elegido:
SPS00


buenos
Pares de lemma/tag posibles:
[(u'bueno', u'AQ0MP0')]
Tag elegido:
AQ0MP0


aires
Pares de lemma/tag posibles:
[(u'aire', u'NCMP000')]
Tag elegido:
NCMP000


tienen
Pares de lemma/tag posibles:
[(u'tener', u'VMIP3P0')]
Tag elegido:
VMIP3P0


esas
Pares de lemma/tag posibles:
[(u'ese', u'DD0FP0'), (u'ese', u'PD0FP000')]
Tag elegido:
DD0FP0


casas
Pares de lemma/tag posibles:
[(u'casa', u'NCFP000'), (u'casar', u'VMIP2S0')]
Tag elegido:
NCFP000


azules
Pares de lemma/tag posibles:
[(u'azul', u'

[[(u'nunca', u'nunca'),
  (u'te', u'te'),
  (u'cre\xed', u'cre\xed'),
  (u'.', u'.')],
 [(u'las', u'el'),
  (u'calles', u'calle'),
  (u'de', u'de'),
  (u'buenos', u'bueno'),
  (u'aires', u'aire'),
  (u'tienen', u'tener'),
  (u'esas', u'ese'),
  (u'casas', u'casa'),
  (u'azules', u'azul'),
  (u'.', u'.')]]